# 01_scrap

- scrapping data from thejewelrymagazine.com and upload to DataPlatform

## Getting data from thejewelrymagazine.com

In [ ]:
pip install scrapy

# Import Library

In [ ]:
import scrapy
import pandas as pd

from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

In [ ]:
class magazine(scrapy.Spider):
    name = "events_jewelrymagazine"
    start_urls = ['https://www.thejewelrymagazine.com/news-trends/show-events/'
    ]
    
    custom_settings = {
        'FEEDS': { 'events_jewelrymagazine.json': { 'format': 'json',}}
        }
        
    
    def parse(self, response):
        links = response.css('h3.entry-title a::attr(href)').extract()
        for link in links:
            eventsLink = 'https://www.thejewelrymagazine.com/news-trends/show-events/' + link
            yield response.follow(link, callback=self.parse_events)

    def parse_events(self, response):
        yield{
            'Title' : response.css('h1.entry-title::text').get().strip(),
            'Publish_time' : response.css('time.entry-date ::text').get(),
            'Description' : response.xpath("//meta[@name='description']/@content")[0].extract(),
            'Link' : response.xpath("//link[@rel='canonical']/@href")[0].extract()
        }

In [ ]:
process = CrawlerProcess()
process.crawl(magazine)
process.start()

# Upload data to DataPlatform

In [ ]:
import requests
from tqdm.auto import tqdm

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 62

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='events_jewelrymagazine.json')